# Figure 2 c

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import Grouse, calc_subspace_proj_error

from matplotlib import cm

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e1/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

p,T_full,n,snr = 1000, 100000, 20, (9., 9.)
Ts = (1000,10000,100000)

run = '_e1_init'
file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) + 'snr' + str(np.int(np.mean(snr)//1)) +  run
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
pars_true = load_file['pars_true'].copy()
idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()

ev_true = np.linalg.eigvals(pars_true['A'])
ev_idx = np.argsort(np.angle(ev_true))
ev_true = ev_true[ev_idx]


def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

clrs = cm.jet(np.linspace(50, 200, len(Ts), dtype=int))
lgnd = []


plt.figure(figsize=(10,4))


"""
plt.subplot(2,2,2)
plt.plot( range(1, n+1), 90 * np.angle(ev_true) / np.pi, '.-', color = 'k', linewidth=2)
plt.hold(True)
plt.subplot(2,2,4)
plt.plot( range(1, n+1), np.abs(ev_true), '.-', color = 'k', linewidth=2)
plt.hold(True)
"""

plt.subplot(1,4,3)
plt.plot(np.real(ev_true), np.imag(ev_true), 'kx', markeredgewidth=5, markersize=10)
plt.hold(True)

plt.subplot(1,4,4)
plt.plot(pars_true['R'], np.arange(1, p+1), color = 'k', linewidth=2)

for Ti in range(len(Ts)):
    T = Ts[Ti]
    print('T = ', str(T))
    
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_final'
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_g']
    traces = load_file['traces']
    traces_g = load_file['traces_g']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    plt.subplot(1,2,1)
        
    plt.plot( range(1, n+1), 90 * principal_angle(pars_true['C'], pars_est['C']), 'o', color=clrs[Ti,:] )
    plt.plot( range(1, n+1), 90 * principal_angle(pars_true['C'], pars_est_g['C']), 'x', color=clrs[Ti,:], 
              markeredgewidth=3)
    #plt.plot( principal_angle(pars_est['C'], pars_est_g['C']), '.', color=clrs[Ti,:] )
    lgnd.append('T = ' + str(T) + ', us')
    lgnd.append('T = ' + str(T) + ', GROUSE')
    
    plt.subplot(1,4,3)
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.angle(ev_est))
    ev_est = ev_est[ev_idx]    
    plt.plot(np.real(ev_est), np.imag(ev_est), 'o', color = clrs[Ti,:])
    angles = np.arange(-np.pi/10, np.pi/10, 0.01)
    plt.plot(np.cos(angles), np.sin(angles))
    
    
    plt.subplot(1,4,4)
    plt.plot(np.sort(pars_est['R']), np.arange(1, p+1), color = clrs[Ti,:], linewidth=1.5)
    
    
    """
    plt.subplot(2,2,2)
    
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.abs(ev_est))
    ev_est = ev_est[ev_idx]

    plt.plot( range(1, n+1), 90 * np.angle(ev_est)  / np.pi, 'o', color = clrs[Ti,:] )
    
    plt.subplot(2,2,4)
    
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.abs(ev_est))
    ev_est = ev_est[ev_idx]

    plt.plot( range(1, n+1), np.abs(ev_est), 'o', color = clrs[Ti,:] )
    """
    
plt.subplot(1,2,1)
plt.box('off')
plt.legend(lgnd, loc=2,frameon=False)
plt.ylabel('principal angle [deg]')
plt.xlabel('latent dim.')
plt.axis([0.5, n+0.5, 0, 90])
plt.title('subpsace estimation')
plt.xticks(range(5, n+1, 5))
plt.yticks(range(0, 91, 30))

plt.subplot(1,4,3)
plt.box('off')
plt.ylabel('imag[\lambda_i]')
plt.xlabel('imag[\lambda_i]')b
plt.axis([0.925, 1, -0.3, 0.3])
plt.title('eigenvalues of dynamics')
plt.xticks([0.93, 0.96, 0.99])
plt.yticks([-0.2, 0, 0.2])
plt.subplot(1,4,3).yaxis.tick_right()
plt.subplot(1,4,3).yaxis.set_label_position("right")


plt.subplot(1,4,4)
plt.box('off')
plt.xlabel('R_ii')
plt.ylabel('observed dim. #i')
plt.axis([0.73, 1.04, 0, p+1])
plt.title('private noise')
plt.xticks(np.arange(0.8, 1.01, 0.1))
plt.yticks([1, 500, 1000])
plt.subplot(1,4,4).yaxis.tick_right()
plt.subplot(1,4,4).yaxis.set_label_position("right")

"""
plt.subplot(2,2,2)
plt.box('off')
plt.ylabel('angle of complex eigenvalue')
plt.xlabel('latent dim.')
plt.axis([0.5, n+0.5, -15, 15])
plt.title('dynamic mode quality I')

plt.subplot(2,2,4)
plt.box('off')
plt.ylabel('abs. of complex eigenvalue')
plt.xlabel('latent dim.')
plt.axis([0.5, n+0.5, 0.9, 1.1])
plt.title('dynamic mode quality II')
"""
plt.savefig(fig_path + 'fig2_C.pdf')
plt.show()
